<a href="https://colab.research.google.com/github/Mandira124/Machine-Learning-Beginner-Projects/blob/main/Multi_class_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

In [51]:
from datasets import load_dataset

# load dataset
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

# convert train split to pandas DataFrame
df = pd.DataFrame(dataset["train"])

# preview
print(df.head())

      id                                               text  label sentiment
0   9536                    Cooking microwave pizzas, yummy      2  positive
1   6135  Any plans of allowing sub tasks to show up in ...      1   neutral
2  17697   I love the humor, I just reworded it. Like sa...      2  positive
3  14182                       naw idk what ur talkin about      1   neutral
4  17840          That sucks to hear. I hate days like that      0  negative


In [42]:
df = df.dropna(subset=['text', 'label'])
X = df['text']
y = df['label']


In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#vectorization
#converting text to numerical features for logistic regression
# for english language common words like 'the', 'is', 'at' are removed
tfidf = TfidfVectorizer(stop_words='english', max_features=20000)

# fit TF-IDF on training data
X_train_tfidf = tfidf.fit_transform(X_train)

# transform test data using the same vocabulary
X_test_tfidf = tfidf.transform(X_test)

model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train_tfidf, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [44]:
#prediction
y_pred = model.predict(X_test_tfidf)

#accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
#confusion matrix
print("confusion matrix:\n",confusion_matrix(y_test, y_pred))
#classification report to understand the perfomance of the model

Accuracy: 0.6558348007043381
confusion matrix:
 [[1071  597  156]
 [ 398 1514  410]
 [ 112  477 1512]]


In [50]:
def predict_sentiment(review_text):

    # transform text to TF-IDF vector
    vec = tfidf.transform([review_text])

    # get probability for each class to choose the class with highest prob, this is usually done in multi-class
    probas = model.predict_proba(vec)[0]

    # predict numeric class
    pred_class = probas.argmax()  # class with highest probability

    # map numeric back to human-readable label
    sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    pred_label = sentiment_map[pred_class]

    return pred_label, probas[pred_class]

# examplesss
print(predict_sentiment("I love this new phone!"))
print(predict_sentiment("This is okay, nothing special."))
print(predict_sentiment("I hate this service!"))


('positive', np.float64(0.8053439812326472))
('neutral', np.float64(0.44693717096096436))
('negative', np.float64(0.8745559183042358))
